# Анализ тональности киноотзывов

В качестве примера мы воспользуемся набором данных, который содержит киноотзывы, оставленые на сайте IMDb (Internet Movie Database)

После распаковки нбор данных представляет собой две отдеьные папки с текстовыми файлами, одна
папка - для обучения, а вторая - для тестирования.

Каждая папка, в свою очередь, содержит две подпапки, одна называется pos, а другая - neg.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import mglearn
from IPython.display import display
plt.rc('font', family = 'Verdana')

In [2]:
!tree -L 2 /Users/viktorkobets/Documents/aclImdb

/Users/viktorkobets/Documents/aclImdb
├── README
├── imdb.vocab
├── imdbEr.txt
├── test
│   ├── labeledBow.feat
│   ├── neg
│   ├── pos
│   ├── urls_neg.txt
│   └── urls_pos.txt
└── train
    ├── labeledBow.feat
    ├── neg
    ├── pos
    ├── unsupBow.feat
    ├── urls_neg.txt
    ├── urls_pos.txt
    └── urls_unsup.txt

6 directories, 11 files


Папка pos содержит все положительные отзывы, каждый отзыв записан в виде отдельного текстового файла, папка neg содержит все отрицательные отзывы и также каждый отзыв представлен в виде отдельного текстового файла. Папка unsup сщдержит данные без отзывов, которые мы не будем использовать, и поэтому просто удаляем.

In [3]:
!rm -r /Users/viktorkobets/Documents/aclImdb/train/unsup

rm: /Users/viktorkobets/Documents/aclImdb/train/unsup: No such file or directory


В библиотеке sklearn есть вспомогательная функция load_files. Она позволяет загрузить файлы, для хранения которых используется такая структура папок, в которой каждая вложенная папка соответствует определенной метке. Сначала мы применим функцию load_files к обучающим данным.

In [4]:
from sklearn.datasets import load_files

reviews_train = load_files('/Users/viktorkobets/Documents/aclImdb/train/')
# load_files возвращает коллекцию, содержащую обучающие тексты и обучающие метки

text_train, y_train = reviews_train.data, reviews_train.target

print(u'Тип text_train: {}'.format(type(text_train)))
print(u'Длина text_train: {}'.format(len(text_train)))
print('text_train[1]:\n{}'.format(text_train[1]))

Тип text_train: <type 'list'>
Длина text_train: 25000
text_train[1]:
Words can't describe how bad this movie is. I can't explain it by writing only. You have too see it for yourself to get at grip of how horrible a movie really can be. Not that I recommend you to do that. There are so many clichés, mistakes (and all other negative things you can imagine) here that will just make you cry. To start with the technical first, there are a LOT of mistakes regarding the airplane. I won't list them here, but just mention the coloring of the plane. They didn't even manage to show an airliner in the colors of a fictional airline, but instead used a 747 painted in the original Boeing livery. Very bad. The plot is stupid and has been done many times before, only much, much better. There are so many ridiculous moments here that i lost count of it really early. Also, I was on the bad guys' side all the time in the movie, because the good guys were so stupid. "Executive Decision" should without a dou

In [5]:
text_train = [doc.replace(b'<br />', b' ') for doc in text_train]

Можно видеть, что объект text_train представляет собой список длиной 25000 элементов, в котором каждый элемент представляет собой строку, содержащую отзыв. Выше был напечатан только отзыв с индексом 1.

Набор данных был собран таким образом, чтобы положительный и отрицательный классы были сбалансированы, поэтому количество строк с положительными отзывами и количество строк с отрицательными отзывами в этом наборе одинаковое.

In [6]:
print(u'Количество примеров на класс (обучение): {}'.format(np.bincount(y_train)))

Количество примеров на класс (обучение): [12500 12500]


In [7]:
# аналогичным образом загружаем тестовые данные
reviews_test = load_files('/Users/viktorkobets/Documents/aclImdb/test/')
text_test, y_test = reviews_test.data, reviews_test.target

print(u'Количество документов в тектовых данных: {}'.format(len(text_test)))
print(u'Количество примеров на класс (тест): {}'.format(np.bincount(y_test)))

Количество документов в тектовых данных: 25000
Количество примеров на класс (тест): [12500 12500]


In [8]:
text_test = [doc.replace(b'<br />', b' ') for doc in text_test]

Задача, которую мы хотим решить, можно сформулировать следующим образом: каждому отзыву нужно присвоить метку "положительный" или "отрицательный" на основе анализа его текста. Это стандартная задача бинарной классификации. Однако текстовые данные представлены в формате, который модель машинного обучения не умеет обрабатывать. Нам нужно преобразовать строковое представление текста в числовое представление, к которому можно будет применить алгоритмы машинного обучения.

## Представление текстовых данных в виде "мешка слов"

Используя это представление, мы удаляем структуру исходного текста, например разбивку на главы и параграфы, знаки препинания,форматирование и лишь подсчитываем частоту встречаемости каждого слова в каждом документе корпуса. Удаление структуры и подсчет частоты каждого слова позволяет получить образное представление текста в виде "мешка слов". Получение из документа представления "мешок слов" включает три следующих этапа.

1. Токенизация. Разбиваем каждый документ на слова, которые в нем встречаются (токены), например с помощью пробелов и знаков пунктуации.

2. Построение словаря. Собираем словарь всех слов, которые появляются в любом из документов, и пронумеровываем их (например в алфавитном порядке).

3. Создание разреженной матрицы. Для каждого документа подсчитываем, как часто каждое из слов, занесенное в словарь, встречается в этом документе.

### Применение модели "мешок слов" к синтетическому набору данных

В библиотеке sklearn модель "мешок слов" реализована в классе CountVectorizer, который и выполняет соответствующее преобразование. Для начала давайте применим класс CountVectorizer к синтетическому набору данных, состоящему из двух примеров, чтобы проиллюстрировать его работу.

In [9]:
bards_words = ['The fool doth think he is wise,',
               'but the wise man knows himself to be a fool']

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(bards_words)

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [11]:
print('Размер словаря: {}'.format(len(vect.vocabulary_)))
print('Содержимое словаря:\n {}'.format(vect.vocabulary_))

Размер словаря: 13
Содержимое словаря:
 {u'fool': 3, u'be': 0, u'he': 4, u'himself': 5, u'wise': 12, u'knows': 7, u'is': 6, u'but': 1, u'to': 11, u'the': 9, u'doth': 2, u'think': 10, u'man': 8}


Словарь состоит из 13 слов, начинается со слова "be" и заканчивается словом "wise". Теперь, чтобы получить представление "мешок слов" для обучающих данных, достаточно вызвать метод trnsform.

In [12]:
bag_of_words = vect.transform(bards_words)
print('bag_of_words: {}'.format(repr(bag_of_words)))

bag_of_words: <2x13 sparse matrix of type '<type 'numpy.int64'>'
	with 16 stored elements in Compressed Sparse Row format>


Чтобы взглянуть на фактическое содержимое разряженной матрицы, мы можем преобразовать ее в "плотный" массив NumPy (который, помимо ненулевых элементов, хранит все нулевые элементы) с помощью метода toarray

In [13]:
print(u'Плотное представление bag_of_words:\n{}'.format(bag_of_words.toarray()))

Плотное представление bag_of_words:
[[0 0 1 1 1 0 1 0 0 1 1 0 1]
 [1 1 0 1 0 1 0 1 1 1 0 1 1]]


### Модель "мешка слов" для киноотзывов

In [14]:
vect = CountVectorizer().fit(text_train)
X_train = vect.transform(text_train)
print(u'X_train:\n{}'.format(repr(X_train)))

X_train:
<25000x74849 sparse matrix of type '<type 'numpy.int64'>'
	with 3431196 stored elements in Compressed Sparse Row format>


In [15]:
# словарь включает 74849 элементов

Существует еще один способ получить доступ к словарю - воспользоваться методом get_feature_name. Он возвращает удобный список, в котором каждый элемент соответствует одному признаку.

In [16]:
feature_names = vect.get_feature_names()
print(u'Количество признаков: {}'.format(len(feature_names)))
print('Первые 20 признаков:\n{}'.format(feature_names[:20]))
print('Признаки с 20010 по 20030:\n{}'.format(feature_names[20010:20030]))
print('Каждый 2000-й признак:\n{}'.format(feature_names[::2000]))

Количество признаков: 74849
Первые 20 признаков:
[u'00', u'000', u'0000000000001', u'00001', u'00015', u'000s', u'001', u'003830', u'006', u'007', u'0079', u'0080', u'0083', u'0093638', u'00am', u'00pm', u'00s', u'01', u'01pm', u'02']
Признаки с 20010 по 20030:
[u'dratted', u'draub', u'draught', u'draughts', u'draughtswoman', u'draw', u'drawback', u'drawbacks', u'drawer', u'drawers', u'drawing', u'drawings', u'drawl', u'drawled', u'drawling', u'drawn', u'draws', u'draza', u'dre', u'drea']
Каждый 2000-й признак:
[u'00', u'aesir', u'aquarian', u'barking', u'blustering', u'b\xeate', u'chicanery', u'condensing', u'cunning', u'detox', u'draper', u'enshrined', u'favorit', u'freezer', u'goldman', u'hasan', u'huitieme', u'intelligible', u'kantrowitz', u'lawful', u'maars', u'megalunged', u'mostey', u'norrland', u'padilla', u'pincher', u'promisingly', u'receptionist', u'rivals', u'schnaas', u'shunning', u'sparse', u'subset', u'temptations', u'treatises', u'unproven', u'walkman', u'xylophonist']
